In [ ]:
"""ДЕТЕКЦІЯ ТРАНСПОРТНИХ ЗАСОБІВ З ПОПЕРЕДНЬОЮ ВЕЙВЛЕТНОЮ ОБРОБКОЮ ТА ЗАСТОСУВАННЯМ ФІЛЬТРА КАЛМАНА"""

import pywt
from ultralytics import YOLO
import time
import numpy as np
import cv2

class KalmanTracker:
    def __init__(self, bbox):
        x = (bbox[0] + bbox[2]) / 2
        y = (bbox[1] + bbox[3]) / 2
        w = bbox[2] - bbox[0]
        h = bbox[3] - bbox[1]

        self.kalman = cv2.KalmanFilter(8, 4)
        # [x, y, w, h, vx, vy, vw, vh]
        self.kalman.transitionMatrix = np.eye(8, dtype=np.float32)
        for i in range(4):
            self.kalman.transitionMatrix[i, i+4] = 1

        self.kalman.measurementMatrix = np.zeros((4, 8), dtype=np.float32)
        self.kalman.measurementMatrix[0, 0] = 1  # x
        self.kalman.measurementMatrix[1, 1] = 1  # y
        self.kalman.measurementMatrix[2, 2] = 1  # w
        self.kalman.measurementMatrix[3, 3] = 1  # h

        self.kalman.processNoiseCov = np.eye(8, dtype=np.float32) * 0.6  #більше шуму => швидше пристосовується
        self.kalman.measurementNoiseCov = np.eye(4, dtype=np.float32) * 10**(-3)  #менше шуму => більше довіряє обчисленням

        self.kalman.statePre = np.array([[x], [y], [w], [h], [0], [0], [0], [0]], dtype=np.float32)
        self.kalman.statePost = self.kalman.statePre.copy()

    def predict(self):
        pred = self.kalman.predict()
        x, y, w, h = pred[0], pred[1], pred[2], pred[3]
        return int(x - w / 2), int(y - h / 2), int(x + w / 2), int(y + h / 2)

    def update(self, bbox):
        x = (bbox[0] + bbox[2]) / 2
        y = (bbox[1] + bbox[3]) / 2
        w = bbox[2] - bbox[0]
        h = bbox[3] - bbox[1]
        measurement = np.array([[x], [y], [w], [h]], dtype=np.float32)
        self.kalman.correct(measurement)


def apply_wavelet_centered(img, wavelet='db6', lambda_=0.5):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    coeffs2 = pywt.dwt2(gray, wavelet)
    _, (H, V, D) = coeffs2
    W = np.sqrt(H ** 2 + V ** 2 + D ** 2)
    W = cv2.resize(W, (img.shape[1], img.shape[0]))
    W_centered = W - np.mean(W)
    img_float = img.astype(np.float32)
    W3 = np.stack([W_centered] * 3, axis=-1)
    result = img_float + lambda_ * W3
    return np.clip(result, 0, 255).astype(np.uint8)


def process_video_with_kalman(model_path, input_video_path, output_video_path, frame_step):
    model = YOLO(model_path)
    cap = cv2.VideoCapture(input_video_path)

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    frame_idx = 0
    trackers = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_idx % frame_step == 0:    #кожен n-й кадр — нова детекція
            wavelet_frame = apply_wavelet_centered(frame.copy())
            results = model.predict(wavelet_frame, conf=0.3, verbose=False)[0]    #більша довіра моделі
            new_trackers = []
            for box in results.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                new_trackers.append(KalmanTracker((x1, y1, x2, y2)))
            trackers = new_trackers
            for tracker in trackers:
                x1, y1, x2, y2 = tracker.predict()
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)    #реальна детекція
        else:
            for tracker in trackers:
                x1, y1, x2, y2 = tracker.predict()
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)    #Kalman прогноз

        out.write(frame)
        frame_idx += 1

    cap.release()
    out.release()
    print(f"Відео збережено")


if __name__ == '__main__':
    input_path='C:/Users/nasti/PyCharmProjects/CourseProject/orig_data/video2.mp4'
    cap = cv2.VideoCapture(input_path)
    if cap.isOpened():
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        duration = total_frames / fps if fps != 0 else 0
    cap.release()
    start_time = time.time()
    process_video_with_kalman(
        model_path='C:/Users/nasti/PyCharmProjects/CourseProject/model.pt',
        input_video_path=input_path,
        output_video_path='C:/Users/nasti/PyCharmProjects/CourseProject/res_with_kalman/res.mp4',
        frame_step=1  #передбачення на кожен n-й кадр
    )
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Час виконання програми: {elapsed_time:.2f} секунд")
    print(f"(FPS) = {fps}")
    print(f"Тривалість відео: {duration:.2f} секунд")